In [ ]:
%pip install pymilvus==2.4.13

In [1]:
from pymilvus import connections

try:
    connections.connect(alias="default", host="127.0.0.1", port="19530")
    print("Connection successful!")
except Exception as e:
    print(f"Connection failed: {e}")


Connection successful!


In [2]:
from pymilvus import utility
collections = utility.list_collections()
print("Available collections:", collections)

# for collection_name in collections:
#   utility.drop_collection(collection_name)
# print("All collections have been dropped.")

# collections = utility.list_collections()
# print("Available collections:", collections)

Available collections: ['Yinon_Azulai_24', 'Yisrael_Eichler_25', 'Walid_Al_Huashla_25', 'Karine_Elharrar_20', 'Moshe_Abutbul_25', 'Yisrael_Eichler_23', 'Karine_Elharrar_25', 'Yuli__Edelstein_15', 'Amir_Ohana_23', 'Yinon_Azulai_20', 'Yisrael_Eichler_16', 'Yisrael_Eichler_20', 'Karine_Elharrar_24', 'Amir_Ohana_20', 'Yuli__Edelstein_21', 'Yuli__Edelstein_23', 'Amir_Ohana_22', 'Amir_Ohana_24', 'Yinon_Azulai_22', 'Yinon_Azulai_25', 'Dan_Illouz_25', 'Yuli__Edelstein_16', 'Karine_Elharrar_23', 'Karine_Elharrar_19', 'Yuli__Edelstein_22', 'Yuli__Edelstein_24', 'Amir_Ohana_21', 'Yinon_Azulai_23', 'Yisrael_Eichler_19', 'Karine_Elharrar_22', 'Zeev_Elkin_18', 'Yuli__Edelstein_20', 'Yuli__Edelstein_25', 'Yisrael_Eichler_22', 'Moshe_Abutbul_24', 'Yuli__Edelstein_17', 'Yuli__Edelstein_19', 'Yinon_Azulai_21', 'Yisrael_Eichler_24', 'Karine_Elharrar_21', 'Yuli__Edelstein_14', 'Amir_Ohana_25', 'Gadi_Eisenkot_25', 'Yisrael_Eichler_17', 'Yisrael_Eichler_21', 'Moshe_Abutbul_23']


In [6]:
elastic_ip = '34.0.64.248:9200'
kibana_ip = '34.0.64.248:5601'
es_username = 'user'
es_password = 'knesset'

from elasticsearch import Elasticsearch

# es = Elasticsearch(f'http://{elastic_ip}',http_auth=(es_username, es_password), timeout=1000000)
# resp = es.search(index="all_features_sentences", body={"query":{"match_all": {}}})
# print("Got %d Hits:" % resp['hits']['total']['value'])
# for hit in resp['hits']['hits']:
#    print("id: %(sentence_id)s: speaker_name: %(speaker_name)s: sentence_text: %(sentence_text)s" % hit["_source"])


In [11]:
from elasticsearch import Elasticsearch

es = Elasticsearch(f'http://{elastic_ip}', basic_auth=(es_username, es_password), request_timeout=500)
data_q =[]
# Query definition
query = {
    "query": {
        "bool": {
            "must": [
                {"match": {"speaker_name": "איתמר בן גביר"}}
            ],
            "filter": {
                "script": {
                    "script": {
                        "source": "doc['sentence_text.keyword'].size() > 0 && doc['sentence_text.keyword'].value.length() > 10"
                    }
                }
            }
        }
    }
}

# Initialize scroll
resp = es.search(index="all_features_sentences", body=query, scroll="2m", size=1000)

# Retrieve the scroll ID and first batch of hits
scroll_id = resp['_scroll_id']
hits = resp['hits']['hits']

total_hits = 0
while total_hits<4000:
    for hit in hits:
      data_q.append("%(sentence_text)s" % hit["_source"])
      #print("%(sentence_text)s" % hit["_source"])

    total_hits += len(hits)

    # Fetch the next batch
    resp = es.scroll(scroll_id=scroll_id, scroll="2m")
    scroll_id = resp['_scroll_id']
    hits = resp['hits']['hits']

print(f"Total results retrieved: {total_hits}")

# Clear the scroll to free resources
es.clear_scroll(scroll_id=scroll_id)


C:\Users\1\AppData\Local\Temp\ipykernel_18052\2487647723.py:24: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  resp = es.search(index="all_features_sentences", body=query, scroll="2m", size=1000)


Total results retrieved: 4000


ObjectApiResponse({'succeeded': True, 'num_freed': 1})

In [7]:
import re

# for i in range(len(data_q)):
#     data_q[i] = re.sub(r'[^א-ת ]', '', data_q[i]).strip()


In [7]:
from sentence_transformers import SentenceTransformer

# Initialize the embedding model
embedding_fn = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Encode the documents
docs = data_q.copy()
vectors = embedding_fn.encode(docs)  # Encode into vectors
print("Dim:", len(vectors[0]), vectors[0].shape)  # Dim: 768


c:\Users\1\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dim: 384 (384,)


In [8]:
data = [
    {"id": i, "vector": vectors[i], "text": docs[i]}
    for i in range(len(vectors))
]
print("Data has", len(data), "entities, each with fields:", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))


Data has 4000 entities, each with fields: dict_keys(['id', 'vector', 'text'])
Vector dim: 384


In [9]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection

# Define the collection schema
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True),
    FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=len(vectors[0])),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=512),
]
schema = CollectionSchema(fields, description="Demo collection for quotes of Miri Regev")

# Create or load the collection
collection_name = "Miri_Regev"
collection = Collection(name=collection_name, schema=schema)


In [10]:
# Milvus expects the data in columns (field-wise)
field_data = [
    [entity["id"] for entity in data],  # IDs
    [entity["vector"] for entity in data],  # Vectors
    [entity["text"] for entity in data],  # Text
]

# Insert the data
res = collection.insert(field_data)
print(f"Inserted {len(field_data[0])} entities into {collection_name}.")


Inserted 4000 entities into Miri_Regev_quotes.


In [11]:
# Create an index

index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}
collection.create_index(field_name="vector", index_params=index_params)
print("Index created.")


Index created.


In [6]:
from pymilvus import list_collections

collections = list_collections()
print("Collections in Milvus:", collections)


Collections in Milvus: []


In [13]:
collection = Collection(name="Miri_Regev_quotes")


In [14]:
collection.load()

# Define a query vector (e.g., embedding for a new query)
query_vector = embedding_fn.encode(["מה דעתה תחבורה ציבורית?"])[0]

# Perform the search
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
results = collection.search(
    data=[query_vector],  # Query vector
    anns_field="vector",
    param=search_params,
    limit=15,  # Number of top matches
    output_fields=["text"],  # Include the text field in results
)

# Display results
for result in results:
    for hit in result:
        print(f"Text: {hit.entity.get('text')}")




Text: לתחבורה הציבורית
Text: מי נוסע בתחבורה ציבורית
Text: והתחבורה הציבורית ב
Text: קח תחבורה
Text: עכשיו התפקיד שלנו זה להוסיף את התדירות של האוטובוסים רכבות בעזרת השם  הרכבת מקריית שמונה ועד אילת מתקדמת  רכבות קלות להביא עוד ועוד כלים של תחבורה ציבורית כדי באמת לשכנע את האזרח לוותר על הרכב ולנסוע בתחבורה הציבורית
Text: אתם מעלים את מחיר הנסיעה בתחבורה הציבורית ב
Text: ממשלה שאומרת שאכפת לה מהפריפריה ומהחלשים ובפועל מעלה ב את מחירי הנסיעה בתחבורה הציבורית
Text: יש כאן בעצם ראייה כוללת שמדברת על יכולת של הנגשה וזמינות לאזרח של תחבורה ציבורית  זמינה נגישה וכמובן גם ירוקה עד כמה שניתן
Text: אנחנו רוצים שבכל מתחמי התחנות אדם יגיע מהבית שלו ברכב דוגלגלי יחנה בתחנה ייסע ליעדו על רכבת או על אוטובוס חשמלי או אוטובוס אחר ויחזור
Text: משרד התחבורה הוא משרד גדול משרד שיש לו אחריות אזרחית על אוויר ים ויבשה על כל הכניסה והיציאה אל הארץ ומהארץ החוצה
Text: נדבר על הרכבת הקלה וגם על הכבדה
Text: אני רוצה לתת עוד בשורה שהיא חשובה מאוד ולהודות כאן גם לחברי מקלב סגן השרה שהבאנו איגום משאבים ועצרנו את על

In [8]:
def retrive_quotes(KNS_name, knesset_number):
    es = Elasticsearch(f'http://{elastic_ip}', basic_auth=(es_username, es_password), request_timeout=500)
    data_q = []

    query = {
        "query": {
            "bool": {
                "must": [
                    {"match": {"speaker_name": KNS_name}},
                    {"match": {"knesset_number": knesset_number}}
                ],
                "filter": {
                    "script": {
                        "script": {
                            "source": "doc['sentence_text.keyword'].size() > 0 && doc['sentence_text.keyword'].value.length() > 30"
                        }
                    }
                }
            }
        }
    }

    # Initialize scroll
    resp = es.search(index="all_features_sentences", body=query, scroll="2m", size=4000)

    # Retrieve the scroll ID and first batch of hits
    scroll_id = resp['_scroll_id']
    hits = resp['hits']['hits']

    while hits:
        for hit in hits:
            data_q.append("%(sentence_text)s" % hit["_source"])
        # Fetch the next batch
        resp = es.scroll(scroll_id=scroll_id, scroll="2m")
        scroll_id = resp['_scroll_id']
        hits = resp['hits']['hits']

    print(f"Total results retrieved: {len(data_q)}")

    # Clear the scroll to free resources
    es.clear_scroll(scroll_id=scroll_id)

    for i in range(len(data_q)):
        data_q[i] = re.sub(r'[^א-ת ]', '', data_q[i]).strip()

    return data_q


In [40]:
retrive_quotes("מירי רגב", "23")

C:\Users\1\AppData\Local\Temp\ipykernel_1936\717346000.py:24: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  resp = es.search(index="all_features_sentences", body=query, scroll="2m", size=4000)


Total results retrieved: 4029


['לא יושבראש הכנסת כשהוא עשה ככה הוא התכוון לקיפי לעצמו',
 'אז תמתין לבית המשפט מה אתה רוצה תמתין לבית המשפט',
 'אולי תדחו    אולי בשבוע הבא תקבלו תשובות',
 'הצעות לסדרהיום יכולות לעלות ואז נענה',
 'קודם כול מקלב יענה לכם בשביל זה הוא סגן שר',
 'וסוףסוף יש לי את היכולת להקשיב למקלב מדבר בשמי',
 'אף אחד לא יגיד מילה רעה על הסגן שלי',
 'ברור לשבת פה ולשמוע אותו משיב על השאילתה',
 'חבר הכנסת סגן השרה מקלב להיות כאן ולשמוע את התשובות שלך',
 'אני כבר אומרת לך שאתה עובר אצלי בציון גבוה מאוד',
 'כל נושא הרכבים האוטונומיים ורכבים חשמליים ואוטובוסים חשמליים  כל הסוגיה הזאת מקבלת אצלנו משקל מאוד מאוד כבד בוודאי בכל מה שקשור בתחבורה הציבורית',
 'אנחנו כולנו מבינים שבעוד חמש שנים לא יהיה אפשר לזוז במדינת ישראל לכן אנחנו חייבים ליצור זמינות של כל מה שקשור לרכב חשמלי ודוגלגלי',
 'אנחנו רוצים שבכל מתחמי התחנות אדם יגיע מהבית שלו ברכב דוגלגלי יחנה בתחנה ייסע ליעדו על רכבת או על אוטובוס חשמלי או אוטובוס אחר ויחזור',
 'יש כאן בעצם ראייה כוללת שמדברת על יכולת של הנגשה וזמינות לאזרח של תחבורה ציבורית  זמי

In [31]:
from elasticsearch import Elasticsearch
import re

def retrive_quotes(KNS_name, knesset_number):

    es = Elasticsearch(f'http://{elastic_ip}', basic_auth=(es_username, es_password), request_timeout=500)
    data_q =[]
    # Query definition
    if knesset_number is not None:
        query = {
            "query": {
                "bool": {
                    "must": [
                        {"match": {"speaker_name": KNS_name}},
                        {"match": {"knesset_number": knesset_number}}
                    ],
                    "filter": {
                        "script": {
                            "script": {
                                "source": "doc['sentence_text.keyword'].size() > 0 && doc['sentence_text.keyword'].value.length() > 30"
                            }
                        }
                    }
                }
            }
        }
    else:
        query = {
            "query": {
                "bool": {
                    "must": [
                        {"match": {"speaker_name": KNS_name}}
                    ],
                    "filter": {
                        "script": {
                            "script": {
                                "source": "doc['sentence_text.keyword'].size() > 0 && doc['sentence_text.keyword'].value.length() > 30"
                            }
                        }
                    }
                }
            }
        }

    # Initialize scroll
    resp = es.search(index="all_features_sentences", body=query, size=8000)
    hits = resp['hits']['hits']

    for hit in hits:
        sentence = hit["_source"].get("sentence_text", "")
        data_q.append(sentence)

    print(f"Total results retrieved: {len(data_q)}")

    # Clean sentences
    for i in range(len(data_q)):
        data_q[i] = re.sub(r'[^א-ת ]', '', data_q[i]).strip()

    return data_q

In [43]:
x = retrive_quotes("משה אבוטבול", 1)

Total results retrieved: 0


C:\Users\1\AppData\Local\Temp\ipykernel_19708\1013205245.py:46: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  resp = es.search(index="all_features_sentences", body=query, size=8000)


In [45]:
x

[]

In [52]:
import csv

# Write the list to a CSV file
with open('quotes.csv', 'w', newline='', encoding='utf-8') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['Quote'])  # Add a header
  for quote in d:
    writer.writerow([quote])

print("List has been written to quotes.csv")

List has been written to quotes.csv
